In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime

In [2]:
dataset = pd.read_csv('AAPL.csv',index_col="Date",parse_dates=True)
#dataset

In [ ]:
# convert column "a" of a DataFrame
# dataset["Close"] = dataset["Close"].str.replace(',', '').astype(float)


In [ ]:
# dataset["Volume"] = dataset["Volume"].str.replace(',', '').astype(float)

In [ ]:
# dataset['Close: 30 Day Mean'] = dataset['Close'].rolling(window=30).mean()
# dataset[['Close','Close: 30 Day Mean']].plot(figsize=(16,6))

In [9]:
# for closing prices
training_set=dataset['Close']
training_set=pd.DataFrame(training_set)

start=1200 #1200
end=200 #200

training_set=training_set[-1*start:-1*end]
realvalue=training_set
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

#training_set_scaled = pd.DataFrame(training_set_scaled)
#print(training_set_scaled.head())

In [ ]:
# # # For multiple inputs

# # dataset.head()
# training_set = dataset[['Close','Open','MACD','RSI EWMA']]
# training_set = pd.DataFrame(training_set)

# start=1200 #1200
# end=200 #200

# training_set=training_set[-1*start:-1*end]
# realvalue=training_set

# # Feature Scaling
# from sklearn.preprocessing import MinMaxScaler
# sc = MinMaxScaler(feature_range = (0, 1))
# training_set_scaled = training_set
# training_set_scaled[['Close','Open','RSI EWMA']] = sc.fit_transform(training_set[['Close','Open','RSI EWMA']])
# #print(training_set_scaled.head())


In [10]:
# Creating a data structure with 60 timesteps and 1 output
xdata= []
ydata =  realvalue['Close']
window=30
length=training_set_scaled.shape[0]-1
#print(training_set_scaled.head())
#column_idx = training_set_scaled.columns.get_loc('Close')

training_set_scaled = np.array(training_set_scaled)

for i in range(window, length):
    xdata.append(training_set_scaled[i-window:i,:])
    #ydata.append(training_set_scaled[i+1, column_idx]) 
xdata, ydata = np.array(xdata), np.array(ydata)

per=0.9 #0.9
split=int(per*len(xdata))
X_test=xdata[split:,:]
Y_test=ydata[split+31:]
realvalue1=realvalue[-len(Y_test):]

X_train=xdata[:split,:]            
y_train=ydata[:split]

# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))


In [11]:
# Part 2 - Building the RNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Initialising the RNN
regressor = Sequential()



Using TensorFlow backend.


In [12]:
# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences =True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')


In [ ]:

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32,shuffle=True)


Epoch 1/100
872/872 [==============================] - 9s 10ms/step - loss: 14181.4918
Epoch 2/100
872/872 [==============================] - 5s 6ms/step - loss: 12658.7508
Epoch 3/100
872/872 [==============================] - 5s 6ms/step - loss: 12115.7934
Epoch 4/100
872/872 [==============================] - 7s 8ms/step - loss: 11763.9265
Epoch 5/100
872/872 [==============================] - 5s 6ms/step - loss: 11412.7504
Epoch 6/100
872/872 [==============================] - 6s 7ms/step - loss: 11104.1439
Epoch 7/100
872/872 [==============================] - 6s 7ms/step - loss: 10791.9833
Epoch 8/100
872/872 [==============================] - 6s 7ms/step - loss: 10492.7661
Epoch 9/100
872/872 [==============================] - 6s 7ms/step - loss: 10215.4515
Epoch 10/100
872/872 [==============================] - 6s 7ms/step - loss: 9944.2741
Epoch 11/100
872/872 [==============================] - 6s 7ms/step - loss: 9667.9535
Epoch 12/100
872/872 [==============================]

In [ ]:
xt=X_test
xt = np.reshape(xt, (xt.shape[0], xt.shape[1], xt.shape[2]))
yt=regressor.predict(xt)
#yt = np.reshape(yt,(yt.shape[0], 1))


#predicted_stock_price = regressor.predict(xt)
predicted_stock_price = yt

In [ ]:
pp=pd.DataFrame(predicted_stock_price)
pp1=pp.shift(-1)
prediction=np.where(pp<pp1,1,-1)

real=pd.DataFrame(Y_test)
real1=real.shift(-1)
real1=np.where(real<real1,1,-1)

from sklearn.metrics import accuracy_score
score=accuracy_score(real1, prediction)
score

In [ ]:
pp = np.array(pp)

data=real
data['pp']=pp

data['ret1']=np.log(data.Close/data.Close.shift(1))# calculating the every day return
data['ret1']=real['ret1'].shift(-1) # bringing the return to the previous day
data['p_sig']=prediction # prediction signal
data['v_p']=data.p_sig*real.ret1 #value prediction retun

# Plot the cumulative returns for single leaf in train dataset
data.v_p.cumsum().plot(figsize=(15,6))

# Plot the cumulative returns for full tree in train dataset
data.ret1.cumsum().plot(figsize=(15,6))
plt.legend(["LSTM retun", "market return"]);


In [ ]:
plt.figure(figsize=(20,5))
plt.plot(data['Close'], color = 'red', label = 'Real AAPL Stock Price')
plt.plot(data['pp'], color = 'blue', label = 'Predicted AAPL Stock Price')
plt.title('AAPL Stock Price Prediction')
#for i in j:
#    plt.axvline(x=i)
plt.xlabel('Time')
plt.ylabel('AAPL Stock Price')
plt.legend()
plt.show()